<a href="https://colab.research.google.com/github/hyrahmaaa/Calculator-Model-Premi/blob/main/Kalkulator_Wt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kalkulator Model Premi**

## Hitung Nilai Fungsi Massa Peluang W_t

In [1]:
import streamlit as st
!pip install streamlit
import pandas as pd
import math

st.set_page_config(
    page_title="Kalkulator Pr{Wt = k} (Distribusi Modifikasi)",
    page_icon="🧮",
    layout="centered",
    initial_sidebar_state="expanded"
)

def calculate_pr_Wt_k(n, p, lambda_val, t_val):
    """
    Menghitung probabilitas Pr{Wt = k} berdasarkan rumus yang diperbarui.
    Mengembalikan DataFrame Pandas.
    """
    q = 1 - p
    results = []

    # Hitung e^(-lambda * t)
    # Pastikan lambda_val * t_val tidak terlalu besar agar tidak terjadi overflow atau underflow
    # jika nilai negatif sangat besar, math.exp akan mendekat0i 0.
    # Jika lambda_val atau t_val sangat besar, e^(-lambda * t) bisa menjadi 0.
    try:
        exp_lambda_t = math.exp(-lambda_val * t_val)
    except OverflowError:
        st.error("Nilai lambda * t terlalu besar menyebabkan OverflowError untuk e^(-lambda * t). Coba nilai yang lebih kecil.")
        return pd.DataFrame(), q, 0 # Return empty DataFrame
    except Exception as e:
        st.error(f"Terjadi kesalahan saat menghitung e^(-lambda * t): {e}")
        return pd.DataFrame(), q, 0

    for k in range(n + 1):
        if k < n:
            # Formula untuk k = 0, 1, 2, ..., (n-1)
            binomial_coefficient = math.comb(n, k)
            prob_k = binomial_coefficient * (q**k) * (p**(n - k)) * exp_lambda_t
        else: # k == n
            # Formula untuk k = n
            prob_k = (q**n * exp_lambda_t) + (1 - exp_lambda_t)

        results.append({'k': k, 'Pr{Wt = k}': prob_k})

    df = pd.DataFrame(results)
    return df, q, exp_lambda_t

st.title("🧮 Kalkulator Probabilitas Pr{Wt = k}")

st.markdown(
    """
    Aplikasi ini menghitung probabilitas $Pr\\{W_t = k\\}$ berdasarkan rumus yang diperbarui:

    $$
    Pr\\{W_t = k\\} =
    \\begin{cases}
        \\binom{n}{k} q^k p^{n-k} (e^{-\\lambda t}), & k = 0, 1, 2, \\dots, (n-1) \\\\
        q^n (e^{-\\lambda t}) + (1-(e^{-\\lambda t})), & k = n.
    \\end{cases}
    $$
    dengan $q = 1 - p$.
    """
)

st.sidebar.header("Input Parameter")

# Input widget untuk n
n_input = st.sidebar.number_input(
    "Masukkan nilai n (jumlah percobaan, bilangan bulat non-negatif):",
    min_value=0,
    step=1,
    value=10, # Default value
    key="n_input"
)

# Input widget untuk p
p_input = st.sidebar.number_input(
    "Masukkan nilai p (probabilitas sukses, antara 0 dan 1):",
    min_value=0.0,
    max_value=1.0,
    step=0.01,
    value=0.5, # Default value
    format="%.2f",
    key="p_input"
)

# Input widget untuk lambda (λ)
lambda_input = st.sidebar.number_input(
    "Masukkan nilai \\(\\lambda\\) (lambda, non-negatif):",
    min_value=0.0,
    step=0.01,
    value=0.1, # Default value
    format="%.2f",
    key="lambda_input"
)

# Input widget untuk t (time)
t_input = st.sidebar.number_input(
    "Masukkan nilai t (waktu, non-negatif):",
    min_value=0.0,
    step=0.1,
    value=5.0, # Default value
    format="%.2f",
    key="t_input"
)


if st.sidebar.button("Hitung Probabilitas"):
    # Validasi Input
    if n_input < 0:
        st.error("Nilai 'n' harus bilangan bulat non-negatif.")
    elif not (0 <= p_input <= 1):
        st.error("Nilai 'p' harus antara 0 dan 1.")
    elif lambda_input < 0:
        st.error("Nilai '\\(\\lambda\\)' harus non-negatif.")
    elif t_input < 0:
        st.error("Nilai 't' harus non-negatif.")
    else:
        st.subheader("Hasil Perhitungan")

        try:
            df_results, q_val, exp_lambda_t_val = calculate_pr_Wt_k(n_input, p_input, lambda_input, t_input)

            if not df_results.empty:
                st.write(f"Parameter: n={n_input}, p={p_input}, q={q_val:.2f}, \u03BB={lambda_input}, t={t_input}")
                st.write(f"Nilai $e^{{-\\lambda t}}$: {exp_lambda_t_val:.10f}")

                st.dataframe(df_results.style.format({'Pr{Wt = k}': "{:.10f}"}))

                total_prob = df_results['Pr{Wt = k}'].sum()
                st.write(f"**Total Probabilitas:** {total_prob:.10f}")

                # Menggunakan math.isclose untuk perbandingan floating point
                if not math.isclose(total_prob, 1.0, rel_tol=1e-9):
                    st.warning(
                        "⚠️ Peringatan: Total probabilitas tidak mendekati 1.0. "
                        "Ini mungkin disebabkan oleh pembulatan atau sifat khusus dari formula distribusi."
                    )
                else:
                    st.success("🎉 Total probabilitas mendekati 1.0.")

        except Exception as e:
            st.error(f"Terjadi kesalahan saat menghitung: {e}")
            st.info("Pastikan input Anda valid dan coba lagi.")

st.sidebar.markdown("---")
st.sidebar.info("Dibuat dengan Streamlit")

2025-06-07 08:50:26.786 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-07 08:50:26.788 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-07 08:50:26.931 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-07 08:50:26.932 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-07 08:50:26.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-07 08:50:26.936 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-07 08:50:26.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())